# Лабораторная 3

Датасет для темы "Поиск работы по резюме"

### Скачали датасет с kaggle

Очистка и анализ:

In [1]:
import pandas as pd
data = pd.read_csv('IT_vacancies_full.csv')

In [2]:
m=data.isnull().sum()
print(m)

Ids                       0
Employer                  0
Name                      0
Salary                    0
From                  33165
To                    38288
Experience                0
Schedule                  0
Keys                      0
Description               0
Area                      0
Professional roles        0
Specializations           0
Profarea names            0
Published at              0
dtype: int64


Видим много пропущенных значений в столбцах From (мин. зп) и To (маск. зп) - мы не сможем заполнить эти данные, поэтому удаляем эти столбцы.

В общем нам не важна зарплата

Так же мы удаляем бесполезные столбцы:  

Salary - наличие зп;
Employer - работодатель;
Published at - когда опубликованно;

In [3]:
data=data.drop(columns=['From','To','Salary','Employer','Published at'])

Удаляем повторы по названиям и по проф. области (чтобы датасет был не такой объемный).

In [4]:
counts = data['Name'].value_counts()
unique_values = counts[counts == 1].index
data = data[data['Name'].isin(unique_values)]

In [5]:
counts = data['Profarea names'].value_counts()
unique_values = counts[counts == 1].index
data = data[data['Profarea names'].isin(unique_values)]

### Датасет (самостоятельно)

Выполняем запрос к API HeadHunter для получения списка вакансий от конкретного работодателя

In [6]:
import numpy as np
import requests
from tqdm import tqdm_notebook
import pandas as pd
r = requests.get('https://api.hh.ru/vacancies?employer_id=665467').json() 
vacancies = r['items']
data2 = pd.DataFrame(vacancies)

Удаляем большую часть датасета, так как есть пропущенные значения, есть неинформативные столбцы.

In [7]:
data2=data2.drop(columns=['premium','department','has_test',
                        'response_letter_required','salary',
                        'type','address','response_url','sort_point_distance',
                       'published_at','created_at','archived','apply_alternate_url',
                       'branding','show_logo_in_search','insider_interview',
                       'url','alternate_url','relations','employer','snippet',
                        'contacts','schedule','working_days','working_time_intervals',
                        'working_time_modes','accept_temporary','fly_in_fly_out_duration',
                        'work_format','working_hours','work_schedule_by_days','night_shifts',
                        'accept_incomplete_resumes','employment_form','internship',
                        'adv_response_url','is_adv_vacancy','adv_context'])

Переименовываем столбцы для того, чтобы соеденить датасеты.

In [8]:
data2.rename(columns={'id': 'Ids'}, inplace=True)
data2.rename(columns={'name': 'Name'}, inplace=True)
data2.rename(columns={'area': 'Area'}, inplace=True)
data2.rename(columns={'professional_roles': 'Professional roles'}, inplace=True)
data2.rename(columns={'experience': 'Experience'}, inplace=True)
data2.rename(columns={'employment': 'Schedule'}, inplace=True)

Добавляем столбцы 

In [9]:
data2['Keys'] = None
data2['Description'] = None
data2['Specializations'] = None
data2['Profarea names'] = None

### Общий датасет (подготовка)

Соединяем датасеты

In [10]:
result = pd.concat([data, data2], ignore_index=True)

Лемматизация 

In [11]:
import re
import pymorphy2
lem = pymorphy2.MorphAnalyzer()
def cls(series: pd.Series) -> pd.Series:
    return series.apply(cl)
def cl(text: str) -> str:
    return clear(lemm(text))
def clear(text: str) -> str:
    return re.sub(r'[^а-яА-ЯёЁ0-9]', ' ', text)
def lemm(text: str) -> str:
    return ' '.join(lem.parse(word)[0].normal_form for word in text.lower().split())

In [12]:
result = result.dropna()

In [13]:
colprocess = [
    'Name',
    'Experience',
    'Schedule',
    'Keys',
    'Description',
    'Area',
    'Professional roles',
    'Specializations',
    'Profarea names'
]

for column in colprocess:
    result[column] = cls(result[column])

In [14]:
result.head()

,Ids,Name,Experience,Schedule,Keys,Description,Area,Professional roles,Specializations,Profarea names
0,49315010,менеджер по ит актив,от 3 до 6 год,полный день,заключение договоров ведение переговоров ...,на данный момент мы искать менеджер по акт...,санкт петербург,руководитель группа разработки,информационный технологии ...,закупки информационный технологии интерн...
1,50105203,оператор геймер на дому,нет опыт,удалённый работа,работа в команде желание учиться и развив...,миссия наш компания улучшение клиентский сер...,санкт петербург,другое,начальный уровень мало опыта системный а...,административный персонал начало карьеры ...
2,49787420,менеджер проект в видеостудия,от 1 год до 3 год,полный день,организаторский навыки работа в команде ...,студия денис золотов в поиск менеджер проект ...,санкт петербург,менеджер по маркетинг и рекламе,менеджер по работа с клиентами менеджер п...,продажи информационный технологии интерн...
3,50102837,менеджер координатор,от 1 год до 3 год,полный день,управление проектами ...,в связь с активный развитие компании приглаша...,санкт петербург,архитектор,строительство недвижимость система автом...,высокий менеджмент информационный техноло...
4,50102682,руководитель телефонный продажа,от 1 год до 3 год,полный день,обучение персонала работа в команде ор...,наш команда объединить общий цель предложить...,санкт петербург,руководитель отдел продаж,телекоммуникации сетевой решения продажа...,административный персонал продажи инфо...


В итоге у нас есть 599 строк